# Figure 6: Convergence Distributions

In [1]:
import os
import json
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np
from scipy import stats
import pandas as pd
from copy import deepcopy

from applefy.statistics import fpf_2_gaussian_sigma, gaussian_sigma_2_fpf, TTest, \
    LaplaceBootstrapTest
from applefy.statistics.bootstrapping import GaussianBootstrapTest
from applefy.utils import get_number_of_apertures

## This Code is about

This Code runs Parametric Bootstrapping for Gaussian and Laplace noise at different separations. The resuls are ploted in Figure 8.

## Create the different Bootstrap models

In [2]:
# -1 for the planet
separation1 = get_number_of_apertures(2*2, 1) - 1
separation2 = get_number_of_apertures(2*8, 1) - 1

In [3]:
gaussian_noise1 = np.random.normal(0, 1, separation1)
gaussian_noise2 = np.random.normal(0, 1, separation2)

laplacian_noise1 = np.random.laplace(0, 1, separation1)
laplacian_noise2 = np.random.laplace(0, 1, separation2)

In [4]:
# the Gaussian models
gaussian_test1 = GaussianBootstrapTest(gaussian_noise1, num_cpus=8)
gaussian_test2 = GaussianBootstrapTest(gaussian_noise2, num_cpus=8)

# the Laplacian models
laplace_test1 = LaplaceBootstrapTest(laplacian_noise1, num_cpus=8)
laplace_test2 = LaplaceBootstrapTest(laplacian_noise2, num_cpus=8)

In [ ]:
# Run the Bootstrapping and save the values of T - Gauss
gaussian_T1 = gaussian_test1.run_bootstrap_experiment(memory_size=5e6,
                                                      num_noise_values=separation1,
                                                      num_draws=1e8,
                                                      approximation_interval=np.linspace(-7, 7, 100000))

gaussian_T2 = gaussian_test2.run_bootstrap_experiment(memory_size=5e6,
                                                      num_noise_values=separation2,
                                                      num_draws=1e8,
                                                      approximation_interval=np.linspace(-7, 7, 100000))

In [ ]:
# Run the Bootstrapping and save the values of T - Laplace
laplace_T1 = laplace_test1.run_bootstrap_experiment(memory_size=5e6,
                                                    num_noise_values=separation1,
                                                    num_draws=1e8,
                                                    approximation_interval=np.linspace(-7, 7, 100000))

laplace_T2 = laplace_test1.run_bootstrap_experiment(memory_size=5e6,
                                                    num_noise_values=separation2,
                                                    num_draws=1e8,
                                                    approximation_interval=np.linspace(-7, 7, 100000))

## Create the Plot

In [ ]:
axis_limit = 7.

In [ ]:
def plot_histogram(axis_in,
                   data_in, 
                   df_in, 
                   double=False):
    
    local_data = pd.DataFrame({"T": data_in})
    
    x = np.linspace(-axis_limit, axis_limit, 1000)
    axis_in.plot(x, stats.t.pdf(x, df=df_in),
                 color=sns.color_palette("colorblind")[1], 
                 lw=3,
                 label="T-Distribution")
    
    # This hack is needed to get the histogram in the Legend
    if double:
        axis_in.plot(x, stats.t.pdf(x, df=df_in),
                     color=sns.color_palette("colorblind")[1], 
                     lw=3,
                     label="T-Distribution")
    
    axis_in.plot(x, stats.norm.pdf(x),
                 color="black",
                 ls="--",
                 lw=2,
                 label="Normal-Distribution")
    
    sns.histplot(data=local_data, 
                 x="T",
                 bins=30, 
                 stat="density", 
                 binrange=(-axis_limit, axis_limit),
                 ax=axis_in,
                 #element="step",
                 legend=True)
    
    axis_in.set_xlim(-axis_limit+0.5, axis_limit-0.5)
    axis_in.set_yscale("log")
    axis_in.yaxis.grid()
    axis_in.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# 1.) Create Plot Layout
fig = plt.figure(constrained_layout=False, figsize=(8, 8))
gs0 = fig.add_gridspec(2, 2)
gs0.update(hspace=0.07, wspace=0.09)

ax_inner_gauss = fig.add_subplot(gs0[0, 0])
ax_outer_gauss = fig.add_subplot(gs0[0, 1], sharex=ax_inner_gauss, sharey=ax_inner_gauss)
ax_inner_laplace = fig.add_subplot(gs0[1, 0], sharex=ax_inner_gauss, sharey=ax_inner_gauss)
ax_outer_laplace = fig.add_subplot(gs0[1, 1], sharex=ax_inner_gauss, sharey=ax_inner_gauss)

# Plot the histograms
idx = np.random.randint(0, gaussian_T1.shape[0]-1, int(10e7)) #10e7
plot_histogram(ax_inner_gauss, gaussian_T1[idx], separation1 - 1, double=True)
plot_histogram(ax_outer_gauss, gaussian_T2[idx], separation2 - 1)
plot_histogram(ax_inner_laplace, laplace_T1[idx], separation1 - 1)
plot_histogram(ax_outer_laplace, laplace_T2[idx], separation2 - 1)

# turn of tick lables
plt.setp(ax_outer_gauss.get_yticklabels(), visible=False)
plt.setp(ax_outer_laplace.get_yticklabels(), visible=False)
plt.setp(ax_inner_gauss.get_xticklabels(), visible=False)
plt.setp(ax_outer_gauss.get_xticklabels(), visible=False)
ax_inner_laplace.tick_params(axis='both', which='major', labelsize=14)
ax_outer_laplace.tick_params(axis='both', which='major', labelsize=14)
ax_inner_gauss.tick_params(axis='both', which='major', labelsize=14)
ax_outer_gauss.tick_params(axis='both', which='major', labelsize=14)

# Lables and Titles
ax_outer_gauss.set_ylabel("")
ax_outer_laplace.set_ylabel("")
ax_inner_gauss.set_xlabel("")
ax_outer_gauss.set_xlabel("")
ax_inner_gauss.set_ylabel("$p(T=x|H_0)$", size=16)
ax_inner_laplace.set_ylabel("$p(T=x|H_0)$", size=16)

ax_inner_laplace.set_xlabel("SNR / Test Statistic", size=16)
ax_outer_laplace.set_xlabel("SNR / Test Statistic", size=16)

#ax1 = ax_inner_gauss.twinx()
#plt.setp(ax1.get_yticklabels(), visible=False)
#ax1.yaxis.set_label_position('left')
ax_inner_gauss.set_ylabel(r"$\hat{F}$ is Gaussian" + "\n" + r"$p\left( T^*=x|H_0 \right) $",
                          fontweight="bold", size=18,  labelpad=12)

#ax2 = ax_inner_laplace.twinx()
#plt.setp(ax2.get_yticklabels(), visible=False)
#ax2.yaxis.set_label_position('left')
ax_inner_laplace.set_ylabel(r"$\hat{F}$ is Laplacian" + "\n" + r"$p\left( T^*=x|H_0 \right) $",
                            size=18, fontweight="bold", labelpad=12)


ax_inner_gauss.set_title("Separation 2 FWHM",
                         fontsize=18, fontweight="bold", y=1.02)
ax_outer_gauss.set_title("Separation 8 FWHM", 
                         fontsize=18, fontweight="bold", y=1.02)

# Create the legend
handles, labels = ax_inner_laplace.get_legend_handles_labels()
handles2, _ = ax_inner_gauss.get_legend_handles_labels()

# For some reason the handles overwite the style and alpha
handles = [handles2[0], handles[0], handles[1],]
labels = ["Bootstrap", labels[0], labels[1],]

handles[0].set_alpha(0.5)
handles[0].set_color(sns.color_palette("colorblind")[0])
handles[0].set_lw(4)

leg1 = fig.legend(handles, labels, 
                  bbox_to_anchor=(-0.05, -0.03), 
                  fontsize=16,
                  loc='lower left', ncol=3)

fig.patch.set_facecolor('white')
plt.savefig("./06_Convergence_distribution_BS.pdf", 
            bbox_extra_artists=(leg1,), 
            bbox_inches='tight')